## Exploring the geographic locations

In [9]:
import folium

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [10]:
from rdflib import Graph

# Create a Graph
g = Graph().parse("rdf/movingImageArchiveEnriched.rdf")

In [11]:
print('##### schema:Place')

# Query the data in g using SPARQL
# This query returns the 'name' of all ``edm:Place`` instances
q = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX schema: <http://schema.org/>
    PREFIX wgs: <http://www.w3.org/2003/01/geo/wgs84_pos#>

    SELECT distinct ?p ?lat ?long ?lbl ?wikidata ?geonames
    WHERE {
        ?p rdf:type schema:Place .
        ?p skos:prefLabel ?lbl .
        ?p wgs:long ?long .
        ?p wgs:lat ?lat .
        ?p owl:sameAs ?wikidata . FILTER ( strstarts(str(?wikidata), "https://www.wikidata.org/wiki/") ).
        ?p owl:sameAs ?geonames . FILTER ( strstarts(str(?geonames), "https://www.geonames.org/") )
    }
"""

##### schema:Place


In [12]:
# Apply the query to the graph and iterate through results
map_circles = folium.Map(location=[55.86,-4.25], tiles="OpenStreetMap", zoom_start=5)

for r in g.query(q):
    idwikidata = r['wikidata']
    lat = r['lat']
    lon = r['long']
    idgeonames = r['geonames']
    label = r['lbl']
    
    text_popup = "Records in <a href='" + idwikidata + "'>" + label + "</a>"

    folium.Circle(
      location=[lat, lon],
      popup=text_popup,
      #radius=float(total)/10,
      color='crimson',
      fill=True,
      fill_color='crimson'
    ).add_to(map_circles)

map_circles